In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel('/content/krasnodar_hack_data.xlsx')
df['date'] = df['date'].apply(lambda x: x[1:-1])
df['date'] = pd.to_datetime(df['date'])
df['floor'] = df['floor'].apply(lambda x: str(x).replace('--', '-'))
df['liter_num'] = df['liter_num'].apply(lambda x: str(x).replace('--', '-'))
df['price_per_m'] = df['price_per_m'].apply(lambda x: x.replace('\xa0', '').replace(' ', '')).astype(int)
df['num_beds'] = df['num_beds'].astype(int)
df['area'] = df['area'].apply(lambda x: str(x).replace(',', '.')).astype(float)
df

,community,liter_num,done_date,developer,floor,area,num_beds,price_per_m,date
0,ЖК «Смородина»,1-5,1 квартал 2023,ООО «АЛЬФА»,1-4,32.95,1,118000,2022-01-25
1,ЖК «Смородина»,1-5,1 квартал 2023,ООО «АЛЬФА»,5-8,32.95,1,119000,2022-01-25
2,ЖК «Смородина»,1-5,1 квартал 2023,ООО «АЛЬФА»,9-12,32.95,1,120000,2022-01-25
3,ЖК «Смородина»,1-5,1 квартал 2023,ООО «АЛЬФА»,13-16,32.95,1,121000,2022-01-25
4,ЖК «Смородина»,1-5,1 квартал 2023,ООО «АЛЬФА»,1-4,35.20,1,117000,2022-01-25
...,...,...,...,...,...,...,...,...,...
1284,ЖК «Role Clef»,nan,2 полугодие 2021,ООО «Уютный дом»,2-12,NaN,1,586000,2023-02-15
1285,ЖК «Role Clef»,nan,2 полугодие 2021,ООО «Уютный дом»,2-12,NaN,2,400000,2023-02-15
1286,ЖК «Role Clef»,nan,2 полугодие 2021,ООО «Уютный дом»,2-12,NaN,3,395000,2023-02-15
1287,ЖК «Мелодия»,nan,дом сдан,NaN,5,NaN,2,94000,2023-02-15


In [ ]:
df.to_csv('krasnodar_data_final.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/krasnodar_hack/krasnodar_data_final.csv')
df = df[~df['area'].isna()]
df = df[~df['done_date'].isna()]


In [ ]:
df_uniq = df.drop(columns=['price_per_m', 'date', 'floor']).drop_duplicates() #.to_csv('unique_communities.csv', index=False)

In [ ]:
df = df.fillna('nan')

In [ ]:
t = df.groupby(['community', 'liter_num', 'done_date', 'developer', 'area', 'num_beds', 'date'])['price_per_m'].mean()

In [ ]:
t = t.reset_index()

In [ ]:
t.community.value_counts()

ЖК «Novella»             53
ЖК «Зеленодар»           47
ЖК «AVrorA»,             42
ЖК «Смородина»           40
ЖК «Fresh»               39
ЖК «The Grand Palace»    21
ЖК «Небо»                20
ЖК «URAL»,               18
ЖК «Спортивный парк»     17
ЖК «Дом на Лаврова»       8
Name: community, dtype: int64

In [ ]:
rep = {
    '4 квартал 2021' : "4 квартал 2021",
    '9 квартал 2021' : "4 квартал 2021",
    '8 квартал 2021' : "4 квартал 2021",
    '7 квартал 2021' : "4 квартал 2021",
    '6 квартал 2021' : "4 квартал 2021",
    '5 квартал 2021' : "4 квартал 2021",
    '18 квартал 2021' : "4 квартал 2021",
    '17 квартал 2021' : "4 квартал 2021",
    '16 квартал 2021' : "4 квартал 2021",
    '14 квартал 2021' : "4 квартал 2021",
    '13 квартал 2021' : "4 квартал 2021",
    '12 квартал 2021' : "4 квартал 2021",
    '11 квартал 2021' : "4 квартал 2021",
    '10 квартал 2021' : "4 квартал 2021",
    '15 квартал 2021' : "4 квартал 2021",
}
kvartal_to_date = {
    '4 квартал 2021': '2021-12-15',
    '2 квартал 2022': '2022-06-15',
    '3 квартал 2022': '2022-09-15',
    '1 квартал 2023': '2023-03-15',
    '2 квартал 2023': '2023-06-15',
    '3 квартал 2023': '2023-09-15',
    '4 квартал 2023': '2023-12-15',
    '1 квартал 2024': '2024-03-15',
    '2 квартал 2024': '2024-06-15',
    '3 квартал 2024': '2024-09-15',
    '4 квартал 2024': '2024-12-15',
    'дом сдан': '1990-12-15'
}

In [ ]:
t['done_date'] = t.done_date.replace(rep)

In [ ]:
t.done_date = t.done_date.replace(kvartal_to_date)

In [ ]:
t.done_date = pd.to_datetime(t.done_date)
t.date = pd.to_datetime(t.date)
t['days_to_done'] = t['done_date'] - t['date']
t['days_to_done'] = t['days_to_done'].apply(lambda x: int(str(x).split()[0]))


In [ ]:
trend = []
for idx, row in t.iterrows():
    if row.date == pd.to_datetime('2022-01-25'):
        trend.append(0)
        continue
    elif row.date == pd.to_datetime('2022-11-06') and row.area == t.iloc[idx-1].area:
        trend.append(row.price_per_m/(t.iloc[idx-1].price_per_m)*100)
        continue
    elif row.date ==  pd.to_datetime('2023-02-15') and row.area == t.iloc[idx-1].area:
        trend.append(row.price_per_m/(t.iloc[idx-2].price_per_m)*100)
        continue
    else:
        trend.append(0)

In [ ]:
t['trend'] = trend

In [ ]:
t['trend'] = t.trend.apply(lambda x: round(x, 2))

In [ ]:
t = t[~((t['trend'] < 100) & (t['trend'] > 50))]

In [ ]:
same_cols = ['community', 'liter_num', 'done_date', 'developer', 'area', 'num_beds']

In [ ]:
new_df = []
for idx, row in t.iterrows():
    if row.date == pd.to_datetime('2022-01-25'):
        i = 1
        if idx+i < t.shape[0]:
            if (t.iloc[idx+i][same_cols] == row[same_cols]).values.sum() == 6:
                row['price_per_m'] = t.iloc[idx+i]['price_per_m']
                row['trend'] = t.iloc[idx+i]['trend']

        new_df.append(row)
    if row.date == pd.to_datetime('2022-11-06'):
        i = 1
        if idx+i < t.shape[0]:
            if (t.iloc[idx+i][same_cols] == row[same_cols]).values.sum() == 6:
                row['price_per_m'] = t.iloc[idx+i]['price_per_m']
                row['trend'] = t.iloc[idx+i]['trend']

        new_df.append(row)

In [ ]:
new_df = pd.DataFrame(new_df)

In [ ]:
new_df = new_df[~(new_df['trend'] == 0)]

In [ ]:
t

,community,liter_num,done_date,developer,area,num_beds,date,price_per_m,days_to_done,trend
0,"ЖК «AVrorA»,",nan,2022-09-15,ООО «АльфаСтройКомплекс»,44.0,1,2022-01-25,187000.0,233,0.00
1,"ЖК «AVrorA»,",nan,2022-09-15,ООО «АльфаСтройКомплекс»,44.0,1,2022-11-06,202000.0,-52,108.02
2,"ЖК «AVrorA»,",nan,2022-09-15,ООО «АльфаСтройКомплекс»,44.0,1,2023-02-15,236600.0,-153,126.52
3,"ЖК «AVrorA»,",nan,2022-09-15,ООО «АльфаСтройКомплекс»,47.0,1,2022-01-25,200750.0,233,0.00
4,"ЖК «AVrorA»,",nan,2022-09-15,ООО «АльфаСтройКомплекс»,47.0,1,2022-11-06,205750.0,-52,102.49
...,...,...,...,...,...,...,...,...,...,...
300,ЖК «Спортивный парк»,7,2022-06-15,ООО «АЛЬФА»,45.0,1,2023-02-15,101800.0,-245,117.01
301,ЖК «Спортивный парк»,8,2022-06-15,ООО «АЛЬФА»,48.0,1,2022-01-25,84000.0,141,0.00
302,ЖК «Спортивный парк»,8,2022-06-15,ООО «АЛЬФА»,48.0,1,2022-11-06,98000.0,-144,116.67
303,ЖК «Спортивный парк»,8,2022-06-15,ООО «АЛЬФА»,51.0,1,2022-01-25,79000.0,141,0.00


In [ ]:
t['date'].unique()

array(['2022-01-25T00:00:00.000000000', '2022-11-06T00:00:00.000000000',
       '2023-02-15T00:00:00.000000000'], dtype='datetime64[ns]')

In [ ]:
test_df = new_df[['area', 'num_beds', 'price_per_m', 'days_to_done', 'community']]
test_df = test_df[test_df['days_to_done'] > -2000]
test_df['community'] = test_df['community'].apply(lambda x: x.strip())

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0xBACED)

In [ ]:
for fold, (train_index, val_index) in enumerate(kfold.split(test_df, test_df['days_to_done'])):
    if fold == 0:
        df_train = test_df.iloc[train_index]
        df_val = test_df.iloc[val_index]


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [ ]:
X = df_train.drop(columns=['price_per_m'])
y = df_train['price_per_m'].values

X_val = df_val.drop(columns=['price_per_m'])
y_val = df_val['price_per_m'].values

In [ ]:
!pip install shap catboost -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.9/575.9 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 12.0 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor, Pool

In [ ]:
regression_params = dict(
    iterations=5000, 
    learning_rate=0.05,
    verbose=100,
    early_stopping_rounds=500,
    loss_function='RMSE',
    eval_metric='MAE',
    task_type='CPU',
    cat_features=['community']
)


In [ ]:
model = CatBoostRegressor(**regression_params).fit(X, y, eval_set=(X_val, y_val))

0:	learn: 37709.1413245	test: 37499.9541219	best: 37499.9541219 (0)	total: 848us	remaining: 4.24s
100:	learn: 9375.1949588	test: 8095.7973147	best: 8082.1061214 (99)	total: 60.6ms	remaining: 2.94s
200:	learn: 6081.2680389	test: 6465.3010380	best: 6465.3010380 (200)	total: 185ms	remaining: 4.41s
300:	learn: 4427.3214133	test: 6545.6179437	best: 6361.4411869 (251)	total: 397ms	remaining: 6.19s
400:	learn: 3395.6664363	test: 6681.1132311	best: 6361.4411869 (251)	total: 551ms	remaining: 6.31s
500:	learn: 2712.4185987	test: 6611.7024042	best: 6361.4411869 (251)	total: 732ms	remaining: 6.58s
600:	learn: 2258.2942629	test: 6603.5144130	best: 6361.4411869 (251)	total: 833ms	remaining: 6.1s
700:	learn: 1912.3338542	test: 6570.2824488	best: 6361.4411869 (251)	total: 993ms	remaining: 6.09s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 6361.441187
bestIteration = 251

Shrink model to first 252 iterations.


In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
y_pred = model.predict(X_val)

1-mean_absolute_percentage_error(y_pred, y_val)

0.9550957107941889

In [ ]:
y_val

array([193000.        , 226700.        , 232200.        , 219600.        ,
       206000.        , 202200.        , 176583.33333333,  96000.        ,
       117000.        , 118200.        , 129540.        , 141500.        ,
       128850.        , 135500.        , 135500.        , 117500.        ,
       113500.        , 108500.        , 186040.        , 183666.66666667,
       106333.33333333, 131033.33333333, 115400.        , 121500.        ,
       109000.        ,  92500.        ])

In [ ]:
y_pred

array([202911.61234791, 236242.42223859, 233670.36797393, 210230.78031406,
       183976.26830126, 200929.81459329, 186790.2239417 , 103177.72064145,
       123926.00722971, 120629.70265349, 132298.84564404, 139909.99354828,
       109067.28903354, 138292.26372966, 140931.85016274, 114561.81108316,
       115840.26418613, 109029.5771292 , 182236.07980921, 179040.32183855,
       112353.82995154, 132299.89543314, 122734.93978517, 131085.9483305 ,
       111778.34988534, 103989.33626743])

In [ ]:
model.predict(pd.DataFrame({'area': [41], 'num_beds': [1], 'days_to_done': [200], 'community':['ЖК «Спортивный парк»']}))

array([118381.91800424])

In [ ]:
pd.DataFrame({'feature_importance': model.get_feature_importance(Pool(X, y, cat_features=['community'])), 
              'feature_names': X.columns}).sort_values(by=['feature_importance'], 
                                                           ascending=False)

,feature_importance,feature_names
3,43.253428,community
2,20.491219,days_to_done
0,19.305202,area
1,16.950151,num_beds


In [ ]:
X_val.head()

,area,num_beds,days_to_done,community
6,49.0,1,233,"ЖК «AVrorA»,"
7,49.0,1,-52,"ЖК «AVrorA»,"
13,52.0,1,-52,"ЖК «AVrorA»,"
22,67.0,2,-52,"ЖК «AVrorA»,"
24,70.0,2,233,"ЖК «AVrorA»,"


In [ ]:
model.predict(X_val.iloc[0])

202911.6123479114

In [ ]:
model.predict(pd.DataFrame({'area': [44], 'num_beds': [1], 'days_to_done': [400], 'community':['ЖК «AVrorA»,']}))

array([197725.156463])

In [ ]:
import pickle

with open('model_pervichka.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
d = test_df.drop(columns=['price_per_m']).drop_duplicates()

In [ ]:
d.num_beds.value_counts()

1    51
2    38
3    34
0     2
Name: num_beds, dtype: int64

In [ ]:
в

,area,num_beds,days_to_done,community
0,44.0,1,233,"ЖК «AVrorA»,"
1,44.0,1,-52,"ЖК «AVrorA»,"
3,47.0,1,233,"ЖК «AVrorA»,"
4,47.0,1,-52,"ЖК «AVrorA»,"
6,49.0,1,233,"ЖК «AVrorA»,"
...,...,...,...,...
293,45.0,1,-52,ЖК «Спортивный парк»
297,41.0,1,-144,ЖК «Спортивный парк»
299,45.0,1,-144,ЖК «Спортивный парк»
302,48.0,1,-144,ЖК «Спортивный парк»


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/krasnodar_hack/krasnodar_data_final.csv')
df = df[~df['area'].isna()]
df = df[~df['done_date'].isna()]

In [ ]:
df = df.loc[df.drop(columns=['price_per_m', 'date', 'floor', 'done_date']).drop_duplicates().index]

In [ ]:
rep = {
    '4 квартал 2021' : "4 квартал 2021",
    '9 квартал 2021' : "4 квартал 2021",
    '8 квартал 2021' : "4 квартал 2021",
    '7 квартал 2021' : "4 квартал 2021",
    '6 квартал 2021' : "4 квартал 2021",
    '5 квартал 2021' : "4 квартал 2021",
    '18 квартал 2021' : "4 квартал 2021",
    '17 квартал 2021' : "4 квартал 2021",
    '16 квартал 2021' : "4 квартал 2021",
    '14 квартал 2021' : "4 квартал 2021",
    '13 квартал 2021' : "4 квартал 2021",
    '12 квартал 2021' : "4 квартал 2021",
    '11 квартал 2021' : "4 квартал 2021",
    '10 квартал 2021' : "4 квартал 2021",
    '15 квартал 2021' : "4 квартал 2021",
}
kvartal_to_date = {
    '4 квартал 2021': '2021-12-15',
    '2 квартал 2022': '2022-06-15',
    '3 квартал 2022': '2022-09-15',
    '1 квартал 2023': '2023-03-15',
    '2 квартал 2023': '2023-06-15',
    '3 квартал 2023': '2023-09-15',
    '4 квартал 2023': '2023-12-15',
    '1 квартал 2024': '2024-03-15',
    '2 квартал 2024': '2024-06-15',
    '3 квартал 2024': '2024-09-15',
    '4 квартал 2024': '2024-12-15',
    'дом сдан': '1990-12-15'
}

df['done_date'] = df['done_date'].replace(rep).replace(kvartal_to_date)

df.done_date = pd.to_datetime(df.done_date)
df.date = pd.to_datetime(df.date)
df['days_to_done'] = df['done_date'] - df['date']
df['days_to_done'] = df['days_to_done'].apply(lambda x: int(str(x).split()[0]))

df = df[df['days_to_done'] > -2000]

In [ ]:
import uuid

In [ ]:
df['id'] = [uuid.uuid4() for i in range(df.shape[0])]

<ipython-input-460-5bf864eb9863>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] = [uuid.uuid4() for i in range(df.shape[0])]


In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df['community'] = df['community'].apply(lambda x: x.strip())

In [ ]:
model.predict(pd.DataFrame({'area': [58.0], 'num_beds': [1], 'days_to_done': [1000], 'community':['ЖК «Novella»']}))

array([135260.44035891])

In [ ]:
df.to_csv('info_final.csv', index=False)

In [ ]:
import tqdm

preds = []
for idx, row in tqdm.tqdm(df.iterrows()):
    subjs = {
        'id':str(row['id']),
        'days_to_done':row['days_to_done'],
        'price_at_moment': round(model.predict(row[model.feature_names_])*row.area)
    }
    
    prices_after = []
    for days in [1000, 700, 500, 300, 200, 100, 50, 0, -50, -100]:
        row['days_to_done'] = days
        pred = round(model.predict(row[model.feature_names_])*row.area)
        
        subjs[f'{days}_to_done'] = pred

        if days < subjs['days_to_done']:
            prices_after.append(pred)

            if max(prices_after) == pred:
                subjs['max_price_after_invest'] = pred
                subjs['increase'] = subjs['max_price_after_invest'] - subjs['price_at_moment']
                subjs['increase_procent'] = round((subjs['increase']/(subjs['price_at_moment']*100))*100*100, 2)

                subjs['days_for_increase'] = subjs['days_to_done']-days

                if subjs['days_for_increase'] >= 365:
                    subjs['status'] = 'long'
                else:
                    subjs['status'] = 'short'

    
    if subjs['increase_procent'] >= 7:
        subjs['risk'] = 'good'
    elif subjs['increase_procent'] <= 0:
        subjs['risk'] = 'bad'
    else:
        subjs['risk'] = 'riskey'


    preds.append(subjs)

153it [00:03, 44.60it/s]


In [ ]:
f = pd.DataFrame(preds)

In [ ]:
f.risk.value_counts()

good      67
riskey    47
bad       39
Name: risk, dtype: int64

In [ ]:
df.id = df.id.astype(str)

In [ ]:
f.merge(df[['community', 'liter_num', 'done_date', 'developer', 'floor', 'area',
       'num_beds', 'days_to_done', 'id']], on='id', how='left').to_csv('merged_stats_final.csv', index=False)

In [ ]:
import json

with open('stats_final.json', 'w') as f:
    json.dump(preds, f)

In [ ]:
cols = ['1000_to_done', '700_to_done',
       '500_to_done', '300_to_done', '200_to_done',
       '100_to_done', '50_to_done', '0_to_done', '-50_to_done',
       '-100_to_done']

In [ ]:
import plotly.express as px

In [ ]:
t = f[f['increase_procent'] > 15][cols].T

In [ ]:
fig = px.line(t)
fig.show()